In [1]:
import os
import sys
sys.path.append('../python')

import pandas as pd
import numpy as np
import yaml
import time
import api


In [2]:
# Embed chapter data
chapter_data = pd.read_csv("../../intermediate_data/los_angeles_chapter_data.csv")
chapter_data = chapter_data.fillna('')
api.embed_and_upsert(chapter_data, verbose=True)

0... {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'main': {'vector_count': 5513}},
 'total_vector_count': 5513}


In [3]:
# Embed article data
article_data = pd.read_csv("../../intermediate_data/los_angeles_article_data.csv")
article_data = article_data.fillna('')
api.embed_and_upsert(article_data, verbose=True)

0... 128... {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'main': {'vector_count': 5513}},
 'total_vector_count': 5513}


In [4]:
# Embed section data
section_data = pd.read_csv("../../intermediate_data/los_angeles_section_data.csv")
section_data = section_data.fillna('')
api.embed_and_upsert(section_data, verbose=True)

0... 128... 256... 384... 512... 640... 768... 896... 1024... 1152... 1280... 1408... 1536... 1664... 1792... 1920... 2048... 2176... 2304... 2432... 2560... 2688... 2816... 2944... 3072... 3200... 3328... 3456... 3584... 3712... 3840... 3968... 4096... 4224... 4352... 4480... 4608... 4736... 4864... 4992... 5120... 5248... {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'main': {'vector_count': 5513}},
 'total_vector_count': 5513}


In [5]:
# Embed passage chunk data
passage_chunk_data = pd.read_csv("../../intermediate_data/los_angeles_passage_chunk_data.csv")
passage_chunk_data = passage_chunk_data.fillna('')
api.embed_and_upsert(passage_chunk_data, verbose=True)

0... 128... 256... 384... 512... 640... 768... 896... 1024... 1152... 1280... 1408... 1536... 1664... 1792... 1920... 2048... 2176... 2304... 2432... 2560... 2688... 2816... 2944... 3072... 3200... 3328... 3456... 3584... 3712... 3840... 3968... 4096... 4224... 4352... 4480... 4608... 4736... 4864... 4992... 5120... 5248... 5376... 5504... 5632... 5760... 5888... 6016... 6144... 6272... 6400... 6528... 6656... 6784... 6912... 7040... 7168... 7296... 7424... 7552... 7680... 7808... 7936... 8064... 8192... 8320... 8448... 8576... 8704... 8832... 8960... 9088... 9216... 9344... 9472... 9600... 9728... 9856... 9984... 10112... 10240... 10368... 10496... 10624... 10752... 10880... 11008... 11136... 11264... 11392... 11520... 11648... 11776... 11904... 12032... 12160... 12288... 12416... 12544... 12672... 12800... 12928... 13056... 13184... 13312... 13440... 13568... 13696... 13824... 13952... 14080... 14208... 14336... 14464... 14592... 14720... 14848... 14976... 15104... 15232... {'dimensi